# Optimisation Results Analysis

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

from src.dataloader import get_commute_data
from src.utils import plot_agenda, preprocess_schedules

In [ ]:
KIND = "driving"
results_dir = Path("../results/question_1_a")

## Load data

In [ ]:
paths = results_dir.glob("*.csv")

files = []
for file_ in paths:
    files.append(pd.read_csv(file_))
df_optimised = pd.concat(files, axis=0)
df_optimised = df_optimised.sort_values(["Heure de début", "Heure de fin"])

In [ ]:
# df_optimised[df_optimised.Prestation != "COMMUTE"].to_csv("../results/question_1_b.csv")

In [ ]:
commute_file_paths = [
    "../data/commute_bicycling_clients.csv",
    "../data/commute_driving_clients.csv",
    "../data/commute_bicycling_care_clients.csv",
    "../data/commute_bicycling_clients_care.csv",
    "../data/commute_driving_care_clients.csv",
    "../data/commute_driving_clients_care.csv",
]

commute_data_df = get_commute_data(commute_file_paths)
caregivers = pd.read_excel("../data/ChallengeXHEC23022024.xlsx", sheet_name=2)

In [ ]:
optimised_sched = preprocess_schedules(
    df_optimised, caregivers, sched="optimised", kind=KIND
)

In [ ]:
schedule = pd.read_excel("../data/ChallengeXHEC23022024.xlsx", sheet_name=0)

discard_list = [
    "ADMINISTRATION",
    "VISITE MEDICALE",
    "FORMATION",
    "COORDINATION",
    "HOMMES TOUTES MAINS",
]

schedule = schedule[~schedule.Prestation.isin(discard_list)]
given_sched = preprocess_schedules(
    schedule, caregivers, sched="given", kind=KIND
)

given_sched = given_sched[given_sched.Date.isin(optimised_sched.Date.unique())]

## Plot given agenda

In [ ]:
all_intervenant_agendas = []

for intervenant_id in optimised_sched["ID Intervenant"].unique():
    # NOTE: change kind here depending on execution above
    intervenant_agenda_commute = plot_agenda(
        intervenant_id, given_sched, commute_data_df, kind=KIND
    )
    all_intervenant_agendas.append(intervenant_agenda_commute)

df_agendas_given = pd.concat(all_intervenant_agendas)

## Plot optimised agenda

In [ ]:
all_intervenant_agendas = []

for intervenant_id in optimised_sched["ID Intervenant"].unique():
    # NOTE: change kind here depending on execution above
    intervenant_agenda_commute = plot_agenda(
        intervenant_id, optimised_sched, commute_data_df, kind=KIND
    )
    all_intervenant_agendas.append(intervenant_agenda_commute)

df_agendas_opt = pd.concat(all_intervenant_agendas)

## Comparison of metrics

In [ ]:
print("Commute time in Jan 24")
print(
    f"Given Schedule (in h): {df_agendas_given.loc[(df_agendas_given['Task'] == 'Commute Time'), 'Commute Time'].sum() / 60:.2f}"
)
print(
    f"Optimised Schedule (in h): {df_agendas_opt.loc[(df_agendas_opt['Task'] == 'Commute Time'), 'Commute Time'].sum() / 60:.2f}"
)

In [ ]:
time_given = (
    df_agendas_given[(df_agendas_given["Task"] == "Commute Time")]
    .groupby(["Date", "ID Intervenant"])["Commute Time"]
    .sum()
    .groupby("Date")
    .mean()
    .mean()
)

time_opt = (
    df_agendas_opt[(df_agendas_opt["Task"] == "Commute Time")]
    .groupby(["Date", "ID Intervenant"])["Commute Time"]
    .sum()
    .groupby("Date")
    .mean()
    .mean()
)

print("Avg. commute time per caregiver per day")
print(f"Given Schedule (in min): {time_given:.2f}")
print(f"Optimised Schedule (in min): {time_opt:.2f}")

In [ ]:
print("Number short downtimes in Jan 24")
print(
    f"Given Schedule: {df_agendas_given.loc[(df_agendas_given['Task'] == 'Wait Time'), 'Wait Time'].count()}"
)
print(
    f"Optimised Schedule: {df_agendas_opt.loc[(df_agendas_opt['Task'] == 'Wait Time'), 'Wait Time'].count()}"
)

In [ ]:
print("Short downtimes in Jan 24")
print(
    f"Given Schedule (in h): {df_agendas_given.loc[(df_agendas_given['Task'] == 'Wait Time'), 'Wait Time'].sum() / 60:.2f}"
)
print(
    f"Optimised Schedule (in h): {df_agendas_opt.loc[(df_agendas_opt['Task'] == 'Wait Time'), 'Wait Time'].sum() / 60:.2f}"
)

In [ ]:
cnt_given = (
    df_agendas_given[(df_agendas_given["Task"] == "Wait Time")]
    .groupby(["Date", "ID Intervenant"])["Wait Time"]
    .count()
    .groupby("Date")
    .mean()
    .mean()
)
cnt_opt = (
    df_agendas_opt[(df_agendas_opt["Task"] == "Wait Time")]
    .groupby(["Date", "ID Intervenant"])["Wait Time"]
    .count()
    .groupby("Date")
    .mean()
    .mean()
)

print("Avg. nr. short downtimes per caregiver per day:")
print(f"Given schedule: {cnt_given:.2f}")
print(f"Optimised schedule: {cnt_opt:.2f}")

In [ ]:
commute_given = df_agendas_given.loc[
    (df_agendas_given["Task"] == "Commute Time")
    & (df_agendas_given["Commute Method"] == "driving"),
    "Commute Meters",
].sum()
commute_opt = df_agendas_opt.loc[
    (df_agendas_opt["Task"] == "Commute Time")
    & (df_agendas_opt["Commute Method"] == "driving"),
    "Commute Meters",
].sum()
print("Commute meters in Jan 24")
print(f"Given Schedule (in km): {commute_given / 1000:.2f}")
print(f"Optimised Schedule (in km): {commute_opt / 1000:.2f}")

## Boxplot for a given day

In [ ]:
opt_03_01 = (
    df_agendas_opt[(df_agendas_opt["Task"] == "Commute Time")]
    .groupby(["Date", "ID Intervenant", "Commute Method"])["Commute Time"]
    .sum()
)
opt_03_01 = opt_03_01.reset_index()
opt_03_01 = opt_03_01[opt_03_01.Date == "2024-01-03"]

In [ ]:
sns.set_theme()
sns.set(font_scale=3, style="white")
plt.figure(figsize=(15, 3))
sns.boxplot(
    data=opt_03_01, x="Commute Time", y="Commute Method", color="#D9D9D9"
)  #
plt.ylabel("")
plt.xlabel("")
plt.xlim(0, 175)

In [ ]:
given_03_01 = (
    df_agendas_given[(df_agendas_given["Task"] == "Commute Time")]
    .groupby(["Date", "ID Intervenant", "Commute Method"])["Commute Time"]
    .sum()
)
given_03_01 = given_03_01.reset_index()
given_03_01 = given_03_01[given_03_01.Date == "2024-01-03"]

In [ ]:
sns.set(font_scale=3, style="white")
plt.figure(figsize=(15, 3))
sns.boxplot(
    data=given_03_01, x="Commute Time", y="Commute Method", color="#D9D9D9"
)  #
plt.ylabel("")
plt.xlabel("")
plt.xlim(0, 175)

In [ ]:
print("Number of cargivers working on 03-01-2024")
print("Given schedule: ", given_03_01["ID Intervenant"].nunique())
print("Optimised schedule: ", opt_03_01["ID Intervenant"].nunique())

## Value counts of short downtimes

In [ ]:
given_03_01 = (
    df_agendas_given[(df_agendas_given["Task"] == "Wait Time")]
    .groupby(["Date", "ID Intervenant", "Commute Method"])["Wait Time"]
    .count()
)
given_03_01 = given_03_01.reset_index()
given_03_01 = given_03_01[given_03_01.Date == "2024-01-03"]

In [ ]:
given_03_01["Wait Time"].value_counts()

In [ ]:
opt_03_01 = (
    df_agendas_opt[(df_agendas_opt["Task"] == "Wait Time")]
    .groupby(["Date", "ID Intervenant", "Commute Method"])["Wait Time"]
    .count()
)
opt_03_01 = opt_03_01.reset_index()
opt_03_01 = opt_03_01[opt_03_01.Date == "2024-01-03"]

In [ ]:
opt_03_01["Wait Time"].value_counts()

## Box plots of commute time

In [ ]:
file_path = "../data/ChallengeXHEC23022024.xlsx"
excel_data = pd.ExcelFile(file_path)
intervenants_df = pd.read_excel(excel_data, sheet_name="intervenants")
intervenants_df["Commute Method"] = intervenants_df["Véhicule personnel"].map(
    {"Oui": "driving", "Non": "bicycling", np.nan: "bicycling"}
)

In [ ]:
daily_commute_sum = df_agendas_given.groupby(
    ["ID Intervenant", "Date"], as_index=False
)["Commute Time"].sum()
daily_commute_sum = daily_commute_sum.merge(
    intervenants_df[["ID Intervenant", "Commute Method"]],
    on="ID Intervenant",
    how="left",
)
daily_commute_sum["ID Intervenant"] = daily_commute_sum[
    "ID Intervenant"
].astype(str)

fig = px.box(
    daily_commute_sum,
    x="ID Intervenant",
    y="Commute Time",
    color="Commute Method",
    title="Total Commute Time per Day per Intervenant - Given Schedule",
    color_discrete_map={"driving": "#013D5C", "bicycling": "#237250"},
)
# fig.update_layout(
#     paper_bgcolor='rgba(0,0,0,0)',
#     plot_bgcolor='rgba(0,0,0,0)',
#     width=1200,
#     height=400,
# )

fig.show()

In [ ]:
daily_commute_sum = df_agendas_opt.groupby(
    ["ID Intervenant", "Date"], as_index=False
)["Commute Time"].sum()
daily_commute_sum = daily_commute_sum.merge(
    intervenants_df[["ID Intervenant", "Commute Method"]],
    on="ID Intervenant",
    how="left",
)
daily_commute_sum["ID Intervenant"] = daily_commute_sum[
    "ID Intervenant"
].astype(str)

fig = px.box(
    daily_commute_sum,
    x="ID Intervenant",
    y="Commute Time",
    color="Commute Method",
    title="Total Commute Time per Day per Intervenant - Optimised Schedule",
    color_discrete_map={"driving": "#013D5C", "bicycling": "#237250"},
)

fig.show()